Copy this notebook to a write-able directory

# Start IPyParallel Cluster

## (recommended) use the ipywigets 

See the code cell bellow and run

## (not recommended) manually start/stop ipycluster

To manually start an ipcluster

```shell
ipcluster start -n <num-workers>
```

To manually stop an ip cluster

```shell
ipcluster stop
```

If the cluster does not stop gracefully use the option `--signal=9`

In [13]:
import os
import subprocess
import shutil

from ipywidgets import IntText, Button, Output, Box, Layout, HTML

def is_running():
    pid_filename = os.path.expanduser('~/.ipython/profile_default/pid/ipcluster.pid')
    
    if not os.path.isfile(pid_filename):
        return False
    
    with open(pid_filename) as f:
        pid = f.read()[:-1]
        
    return os.path.isdir(f'/proc/{pid}')

number_workers = IntText(value=1, description='Workers', disabled=False)
number_workers.layout.visibility = 'hidden' if is_running() else 'visible'
action_button = Button(description='stop cluster' if is_running() else 'start cluster')
output = Output()

def on_button_clicked(b):
    status = is_running()
    
    if status: # is running
        command = [shutil.which('ipcluster'), 'stop']
    else:
        command = [shutil.which('ipcluster'), 'start', '-n', str(number_workers.value)]
        
    with output:
        print(command)    
    
    number_workers.layout.visibility = 'hidden' if not status else 'visible'
    b.description = 'stop cluster' if not status else 'start cluster'
    subprocess.Popen(' '.join(command), shell=True)

action_button.on_click(on_button_clicked)

Box([
    HTML('<h1>IPyParallel Cluster</h1>'),
    number_workers,
    action_button,
    output
], layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
))

Box(children=(HTML(value='<h1>IPyParallel Cluster</h1>'), IntText(value=1, description='Workers', layout=Layou…

# Run IPyParallel Computation

In [15]:
import ipyparallel
print(ipyparallel.__version__)

client = ipyparallel.Client()

6.3.0


In [16]:
client.ids

[0]

In [17]:
def slow_power(x, i=5):
    import time
    time.sleep(2)
    return pow(x, i)

In [18]:
result = client[:].apply(slow_power, 5, 5)

In [19]:
result.result()

[3125]